In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
df = pd.read_csv('data.csv')

In [3]:
# Throw away the usernames
df = df.iloc[:, 1:]

In [4]:
df

,Artista 1,Artista 2,Artista 3,Artista 4,Artista 5,Artista 6,Artista 7,Artista 8,Artista 9,Artista 10,...,Artista 21,Artista 22,Artista 23,Artista 24,Artista 25,Artista 26,Artista 27,Artista 28,Artista 29,Artista 30
0,Billie Eilish,Melanie Martinez,Taylor Swift,Eminem,NF,Hollywood Undead,twenty one pilots,Halsey,Olivia Rodrigo,360,...,Nessa Barrett,nicole amoroso,The Neighbourhood,Ed Sheeran,Harry Styles,The Weeknd,James Arthur,One Direction,Panic! at the Disco,XXXTENTACION
1,Billie Eilish,Nessa Barrett,XXXTENTACION,Chris Grey,Paramore,Maggie Lindemann,Lely45,Artemas,Olivia Rodrigo,Ariana Grande,...,Jerry Heil,Big Baby Tape,Morgenstern,Pharaoh,Our Last Night,Nothing But Thieves,Pussykiller,Discollusion,Yeat,INSTASAMKA
2,Billie Eilish,Beyoncé,Doja Cat,Kali Uchis,Ariana Grande,Megan Thee Stallion,Anitta,Nicki Minaj,SZA,Urias,...,Lady Gaga,Melanie Martinez,Harry Styles,The Neighbourhood,Dua Lipa,Pabllo Vittar,Lil Nas X,Sia,Ashe,GloRilla
3,Billie Eilish,Post Malone,Marshmello,Luísa Sonza,bbno$,Taylor Swift,Harry Styles,Trevor Daniel,d4vd,Labrinth,...,Diogo Defante,Lana Del Rey,The Kid LAROI,Iggy Azalea,SadBoyProlific,Oliver Tree,M83,League of Legends,7 Minutoz,Hiosaki
4,Billie Eilish,Chase Atlantic,The Weeknd,Lana Del Rey,The Neighbourhood,Labrinth,Montell Fish,Isabel LaRosa,Nessa Barrett,Doja Cat,...,Nick Alexandr,girl in red,Halsey,SYML,d4vd,Lil Peep,Nicki Minaj,Dove Cameron,Pierce the Veil,Odetari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,blink-182,Red Hot Chili Peppers,Ramones,Foo Fighters,+44,Green Day,Sum 41,The Offspring,Box Car Racer,Weezer,...,Arctic Monkeys,Two Door Cinema Club,Blur,Mac DeMarco,Jimmy Eat World,The Rare Occasions,Simple Plan,Journey,The All-American Rejects,Good Charlotte
5500,blink-182,Linkin Park,Foo Fighters,Paramore,Taylor Swift,+44,Fall Out Boy,Box Car Racer,Primus,Sum 41,...,My Chemical Romance,Rise Against,The Smashing Pumpkins,Angels & Airwaves,Green Day,Nirvana,Bob Dylan,Coldplay,Tool,Phoebe Bridgers
5501,Thundercat,TWICE,st47ic,Playboi Carti,NewJeans,xaviersobased,Joji,Pierce the Veil,twikipedia,5L,...,Mac DeMarco,glaive,Lil Uzi Vert,Lil Peep,Kanye West,Yandere,kkbutterfly27xx,Cochise,i9bonsai,PinkPantheress
5502,Gorillaz,Thundercat,Blur,beabadoobee,Tame Impala,The Kunts,The Beatles,Arctic Monkeys,Madness,Metric,...,Damon Albarn,Penny Bank,Burger King,DANGERDOOM,TV Girl,Childish Gambino,Freddie Gibbs,King Geedorah,Ryan Gosling,Beck


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5504 entries, 0 to 5503
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Artista 1   5504 non-null   object
 1   Artista 2   5504 non-null   object
 2   Artista 3   5504 non-null   object
 3   Artista 4   5504 non-null   object
 4   Artista 5   5504 non-null   object
 5   Artista 6   5504 non-null   object
 6   Artista 7   5504 non-null   object
 7   Artista 8   5504 non-null   object
 8   Artista 9   5504 non-null   object
 9   Artista 10  5504 non-null   object
 10  Artista 11  5504 non-null   object
 11  Artista 12  5504 non-null   object
 12  Artista 13  5504 non-null   object
 13  Artista 14  5504 non-null   object
 14  Artista 15  5504 non-null   object
 15  Artista 16  5504 non-null   object
 16  Artista 17  5504 non-null   object
 17  Artista 18  5504 non-null   object
 18  Artista 19  5504 non-null   object
 19  Artista 20  5504 non-null   object
 20  Artista 

In [5]:
# Concatenate all the unique artists
all_artists = pd.concat([df[coluna] for coluna in df.columns]).unique()

In [7]:
print(len(all_artists))

19560


Web-scraping

In [8]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import quote, quote_plus
import json
import time
from requests.exceptions import ConnectionError, Timeout, HTTPError

Function to add to the json, and after create the mongodb database

In [9]:
def add_json(artista, url_artista, url_album, album_name, file_path ='Artistas.json'):
    try:
        # Try to open and load the existing JSON data
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError):
        # If the file doesn't exist or is empty/invalid, initialize an empty dictionary
        data = {}

    data[artista] = {
        "album_name": album_name,
        "artist_img": url_artista,
        "url_imagem_album": url_album,
    }
    
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    

Function to retry, if some connection error occurs

In [10]:
def fetch_url_with_retries(url, retries=5, backoff_factor=0.3, timeout=5):
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            return response
        except (ConnectionError, Timeout) as e:
            print(f"Error trying to get the URL: {e}. Trying again... ({attempt+1}/{retries})")
            time.sleep(backoff_factor * (2 ** attempt))  # Exponential backoff
        except HTTPError as e:
            if e.response.status_code == 404:
                print(f"Erro 404: for {url}. Skippping.")
                pass
            else:
                print(f"HTTP error: {e}. Trying again... ({attempt+1}/{retries})")
                time.sleep(backoff_factor * (2 ** attempt))  # Exponential backoff
    print(f"Failed to get the url after {retries} retries: {url}")
    return None

In [11]:
file_path = 'Artistas.json'

When the add data stopped for some motive, get the json again and start doing where stopped

In [12]:
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        existing_data = json.load(file)
except (FileNotFoundError, json.JSONDecodeError):
    existing_data = {}

Add the data to the json

In [13]:
try:
    for artist in all_artists:
        if artist in existing_data:
            print(f"Artista {artist} já processado. Pulando...")
            continue
        encoded_artist = quote_plus(artist)
        url = f"https://www.last.fm/music/{encoded_artist}/+images"
        print(f"Tentando acessar a URL: {url}")
        response = fetch_url_with_retries(url)
        if response is None:
            continue
        soup = BeautifulSoup(response.content, "html.parser")
        artist_img = soup.find('li', class_='image-list-item-wrapper')
        if artist_img and artist_img.img:
            artist_img = artist_img.img['src']
        else:
            artist_img = "No image found"


        url2 = f"https://www.last.fm/music/{encoded_artist}/+albums"
        print(f"Tentando acessar a URL: {url2}")
        response2 = fetch_url_with_retries(url2)
        if response is None:
            continue
        soup = BeautifulSoup(response2.content, "html.parser")
        album_img = soup.find('span', class_='resource-list--release-list-item-image cover-art')
        if album_img and album_img.img:
            album_img = album_img.img['src']
        else:
            album_img = "No image found"
        album_name = soup.find('a', class_='link-block-target')
        if album_name:
            album_name = album_name.text
        else:
            album_name = "No album name found"
        add_json(artist, artist_img, album_img, album_name)
except Exception as e:
    print(f"Failed to get the info, {e}")
    pass

Artista Billie Eilish já processado. Pulando...
Artista Madison Beer já processado. Pulando...
Artista Kendrick Lamar já processado. Pulando...
Artista Frank Ocean já processado. Pulando...
Artista Kanye West já processado. Pulando...
Artista Taylor Swift já processado. Pulando...
Artista The Weeknd já processado. Pulando...
Artista Yameii Online já processado. Pulando...
Artista Madonna já processado. Pulando...
Artista TV Girl já processado. Pulando...
Artista Joji já processado. Pulando...
Artista YABUJIN já processado. Pulando...
Artista Drake já processado. Pulando...
Artista Destroy Lonely já processado. Pulando...
Artista Coldplay já processado. Pulando...
Artista Sabrina Carpenter já processado. Pulando...
Artista Ariana Grande já processado. Pulando...
Artista BTS já processado. Pulando...
Artista Tate McRae já processado. Pulando...
Artista Lana Del Rey já processado. Pulando...
Artista Leah Kate já processado. Pulando...
Artista LE SSERAFIM já processado. Pulando...
Artista 

In [14]:
print(all_artists[-1])
#check if the last artist is the same that in the url

Dope D.O.D.
